In [1]:
import pandas as pd
from viasegura import ModelLabeler
import numpy as np
import os
from pathlib import Path
import tensorflow as tf
from tqdm.notebook import tqdm

### Espacio descriptivo por cada seccion
***

#### Descripcion
Viasegura es una libreria para etiquetar algunos de los atributos de seguridad vial de la metodologia iRAP. Los atributos especificos que pueden ser etiquetados los encontramos en la libreria en la url:

https://github.com/EL-BID/VIAsegura

Hasta el momento hay 15 modelos desarrollados para etiquetar en total 17 atributos (dado que 2 modelos funcionan para dos atributos distintos.

#### Alcance

Para ejecutar el sistema y entender el alcance de aplicacion de estos modelos por favor referirse al manual ubicado en la url:

https://github.com/EL-BID/VIAsegura/tree/main/viasegura/manuals

En esa seccion encontraran el contexto general y particular bajo el que se puede utilizar el software

### GPU Configuration
***

Como saben los modelos de IA utilizan procesamiento en paralelo y en la mayoria de los casos se requiere procesamiento utilizando una gpu. 

Para que el sistema funcione eficientemente, se prevee que el usuario configura la gpu de la siguiente forma con el objetivo de que los modelos cuenten con los recursos necesarios para su correcta ejecucion.

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:   
    try:    
        tf.config.experimental.set_virtual_device_configuration(gpus[0], 
                                                                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096+2048),])    
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')    
        print(len(gpus), "Physical GPU,", len(logical_gpus), "Logical GPUs")  
    except RuntimeError as e:
        print(e)
device = logical_gpus[0].name        

1 Physical GPU, 1 Logical GPUs


### Inputs
***

El sistema funciona utilizando imagenes o videos, estos videos deben estar ordenados

In [9]:
master_input = 'data/Visor Datos Brasil 2015/Brasil-2015/Visor Datos Brasil 2015/IRAP/' #<Ubicacion de las imagenes a ser utilizadas>
folders_check = ['CAM CEN']
BATCH_SIZE = 32

### Functions
***

In [10]:
def load_image(routes):
    imgs = np.array([tf.image.decode_image(tf.io.read_file(str(route))).numpy() for route in routes])
    return imgs

### Load Images
***

In [11]:
folders_master = os.listdir(master_input)
cam_lat_der_paths = []
route_name = []
image_name = []
for route_1 in tqdm(folders_master):
    folders_route_1 = [item for item in os.listdir(master_input+route_1) if len(item.split('.'))==1]
    for route_2 in folders_route_1:
        for fold in folders_check:
            archives = os.listdir(master_input+route_1+'/'+route_2+'/'+fold)
            cam_lat_der_paths += [master_input+route_1+'/'+route_2+'/'+fold+'/'+item for item in archives]
            route_name += [route_1+route_2 for i in range(len(archives))]
            image_name += [it.upper() for it in archives]    

  0%|          | 0/178 [00:00<?, ?it/s]

In [12]:
df_paths = pd.DataFrame({
    'image_name':image_name,
    'route':route_name,
    'path':cam_lat_der_paths
})
df_paths['number_frame'] = list(map(lambda x: (int(x.split('.')[0].split('_')[0])-1)//5, df_paths['image_name'].values))

In [13]:
df_paths_groups = df_paths.groupby(['route','number_frame']).aggregate({
    'image_name':'first',
    'path':[list,'count']
}).reset_index()
df_paths_groups.columns = [col[0]+'_'+col[1] if col[0]=='path' else col[0] for col in df_paths_groups.columns]

#### Load Image example

In [22]:
group = 1
routes = df_paths_groups.path_list.values
images = []
for group in routes[:5]:
    images+= list(load_image(group))

In [27]:
images = np.array(images)

In [40]:
images[0].shape

(1224, 1624, 3)

### Create labeler 
***

Frontal Filters = ['delineation', 'street_lighting', 'carriageway', 'service_road', 'road_condition', 'skid_resistance', 'upgrade_cost', 'speed_management', 'bicycle_facility', 'quality_of_curve', 'vehicle_parking', 'property_access_points']

In [8]:
labeler = ModelLabeler(model_type = 'frontal',device = device, model_filter = ['delineation','street_lighting','carriageway'])

Configuration Loaded
Loaded model "delineation"
Loaded model "carriageway"
Loaded model "street_lighting"
You have succesfully load 3 models on the category "frontal"


In [42]:
labeler.__dict__

{'downloader': <viasegura.downloader.Downloader at 0x249068c2e80>,
 'model_filter': ['delineation', 'street_lighting', 'carriageway'],
 'system_path': WindowsPath('C:/Users/Jose_Marquez/.conda/envs/via_seg/lib/site-packages/viasegura'),
 'config_path': WindowsPath('C:/Users/Jose_Marquez/.conda/envs/via_seg/lib/site-packages/viasegura/config.json'),
 'model_type': 'frontal',
 'device': '/device:GPU:0',
 'verbose': 0,
 'models_route': WindowsPath('models/models_artifacts'),
 'models': ['delineation', 'carriageway', 'street_lighting'],
 'classes': {'delineation': {0: 'Adequate', 1: 'Poor'},
  'carriageway': {0: 'Undivided road', 1: 'Carriageway of a divided road'},
  'street_lighting': {0: 'Not present', 1: 'Present'}},
 'thresholds': {'delineation': None,
  'carriageway': None,
  'street_lighting': None},
 'model_class': {'delineation': 'softmax',
  'carriageway': 'softmax',
  'street_lighting': 'softmax'},
 'model': <keras.engine.functional.Functional at 0x2490a8f2320>}

### Processing
***

In [51]:
results = labeler.get_labels(images)

100%|████████████████████████████████████████████| 3/3 [00:01<00:00,  2.55it/s]


### Resultados
***

In [52]:
results

{'raw_predictions': {'delineation': array([[9.9913591e-01, 8.6407369e-04],
         [9.9995983e-01, 4.0145114e-05],
         [9.9996364e-01, 3.6416670e-05],
         [9.9713433e-01, 2.8656323e-03],
         [9.5558089e-01, 4.4419128e-02],
         [3.2194006e-01, 6.7805994e-01]], dtype=float32),
  'carriageway': array([[3.2518181e-04, 9.9967480e-01],
         [3.9423915e-04, 9.9960583e-01],
         [8.5993286e-04, 9.9914002e-01],
         [7.7646822e-03, 9.9223524e-01],
         [1.0266295e-02, 9.8973376e-01],
         [1.8241487e-01, 8.1758511e-01]], dtype=float32),
  'street_lighting': array([[1.7641375e-02, 9.8235863e-01],
         [7.1303762e-04, 9.9928695e-01],
         [1.4509552e-02, 9.8549044e-01],
         [1.8323507e-02, 9.8167646e-01],
         [1.1323477e-02, 9.8867655e-01],
         [7.0182723e-01, 2.9817280e-01]], dtype=float32)},
 'numeric_class': {'delineation': array([0, 0, 0, 0, 0, 1], dtype=int64),
  'carriageway': array([1, 1, 1, 1, 1, 1], dtype=int64),
  'street_l

In [31]:
clasification = results['clasification']
numeric_class = results['numeric_class']
raw_predictions = results['raw_predictions']